# 4章 ニューラルネットワークの学習

ここで言う「学習」とは、訓練データから最適な重みパラメータの値を自動で獲得すること

<img  src="images/K34SRVI5L1W6BKEPF9XGQEFLG4XM4VBL.png"/>

## 4.1 データから学習する

### 4.1.1 データ駆動

* ニューラルネットワークの利点は、すべての問題を同じ流れで解くことができるということ
    * 手書き文字や人やモノによって異なる特徴量を自動的に抽出する

### 4.1.2 訓練データとテストデータ

* 訓練データ（教師データ）を使って最適なパラメータを探索
* テストデータを使って訓練したモデルの実力を評価
* 特定のデータセットに過度に対応した状態を過学習(overfitting)という

## 4.2 損失関数

* 損失関数(loss function): ニューラルネットワークの性能の"悪さ"を示す指標
    * この値が最小になるようなパラメータを探索する

### 4.2.1 2乗和誤差

$$
    E = \frac{1}{2} \sum_k (y_k - t_k)^2
$$

* $y_k$: ニューラルネットワークの出力
* $t_k$: 教師データ
* $k$: データの次元数

In [2]:
import numpy as np

In [4]:
def mean_squared_error(y, t):
    return 0.5 * np.sum((y-t)**2)

In [8]:
# 「2」を正解とする（one-hot表現）
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

In [6]:
# 例1：「2」の確率が最も高い場合（0.6）
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]

mean_squared_error(np.array(y), np.array(t))

0.097500000000000031

In [7]:
# 例2：「7」の確率が最も高い場合（0.6）
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]

mean_squared_error(np.array(y), np.array(t))

0.59750000000000003

### 4.2.2 交差エントロピー誤差

$$
    E = - \sum_k t_k \log y_k
$$

In [9]:
def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t * np.log(y + delta))

`np.log(0)`がマイナス無限大となって計算を続けられなくなるため、微小な値`delta`を足している。

In [10]:
# 「2」を正解とする（one-hot表現）
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

In [11]:
# 例1：「2」の確率が最も高い場合（0.6）
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]

cross_entropy_error(np.array(y), np.array(t))

0.51082545709933802

In [12]:
# 例2：「7」の確率が最も高い場合（0.6）
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]

cross_entropy_error(np.array(y), np.array(t))

2.3025840929945458

### 4.2.3 ミニバッチ学習

$$
    E = - \frac{1}{N} \sum_n \sum_k t_{nk} \log y_{nk}
$$

* $N$: 訓練データの個数

$N$個のデータを使って平均の損失関数を求めることで、データ個数によらない統一した指標になる。また、大量のデータすべてを使わなくても、一部のデータ（ミニバッチ）を使って、全体の近似値とすることができる。

In [14]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = \
    load_mnist(normalize=True, one_hot_label=True)

print(x_train.shape) # (60000, 784)
print(t_train.shape) # (60000, 10)

(60000, 784)
(60000, 10)


60000個のデータから10個をランダムに選ぶ。

In [15]:
train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

In [16]:
print(x_batch.shape)
print(t_batch.shape)

(10, 784)
(10, 10)


In [18]:
np.random.choice(60000, 10)

array([  406,  6851, 53632, 35722, 22802, 14257, 27413,  5313, 43132, 29410])

### 4.2.4 ［バッチ対応版］交差エントロピー誤差の実装

データ数が1個でもN個でも対応できるようにした交差エントロピー誤差の実装

In [19]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    batch_size = y.shape[0]
    return -np.sum(t * np.log(y)) / batch_size

教師データ`t`がone-hot表現ではなくラベル（「2」や「7」といった値）として与えられた場合

In [20]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t])) / batch_size

In [21]:
np.arange(10)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

```python
t = [2, 7, 0, 9, 4]
y[np.arange(batch_size), t]
# => [y[0,2], y[1,7], y[2,0], y[3,9], y[4,4]]
```

one-hot表現で`t`の値が0となる要素は、交差エントロピー誤差も0となるので無視してよい。